<a href="https://colab.research.google.com/github/rayshee254/PLP-FINAL-PROJECT/blob/main/Audio_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 🎯 Fixed Audio Processor with Auto Port Selection
# Solves the port conflict issue automatically

# Step 1: Import libraries
import gradio as gr
import whisper
from transformers import pipeline
import re
import io
from datetime import datetime
import os
import socket

# Step 2: Find an available port automatically
def find_available_port(start_port=7860, max_attempts=10):
    """Find an available port starting from the specified port"""
    for port in range(start_port, start_port + max_attempts):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('localhost', port))
                return port
        except OSError:
            continue
    return start_port  # Fallback to original port

# Step 3: Set cache directories
os.environ['HF_HOME'] = '/content/huggingface_cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/huggingface_cache'

# Step 4: Load models (already loaded from your previous run)
print("✅ Models already loaded from previous run!")
print("🔍 Reusing existing models to save time...")

# Get the existing models (they should already be in memory)
try:
    transcriber = whisper.load_model("small")
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6", device=0)
    print("🎯 Models successfully reused!")
except:
    print("⚠️ Could not reuse models, loading fresh...")
    transcriber = whisper.load_model("small")
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6", device=0)

# Step 5: Processing functions (unchanged)
def chunk_text(text, max_words=400):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

def process_audio(audio_file, language="en"):
    if audio_file is None:
        return "", "", "❌ Please upload an audio file first"

    try:
        # Transcribe audio
        result = transcriber.transcribe(audio_file, language=language)
        transcript_raw = result["text"]
        transcript = "\n".join(re.split(r'(?<=[.!?]) +', transcript_raw))

        # Summarize in chunks
        chunks = chunk_text(transcript_raw)
        summaries = []

        for chunk in chunks:
            input_length = len(chunk.split())
            max_len = min(142, max(30, int(input_length * 0.6)))
            min_len = min(30, max_len)

            summary_text = summarizer(
                chunk,
                max_length=max_len,
                min_length=min_len,
                do_sample=False
            )[0]['summary_text']
            summaries.append(summary_text)

        summary = "\n\n".join(summaries)
        return transcript, summary, "✅ Processing complete!"

    except Exception as e:
        return "", "", f"❌ Error: {str(e)}"

def download_results(transcript, summary):
    if not transcript and not summary:
        return None, "❌ No content to download"

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"transcription_summary_{timestamp}.txt"

    content = f"""TRANSCRIPTION & SUMMARY REPORT
    Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    ===================================================

    FULL TRANSCRIPT:
    {transcript if transcript else "No transcript available"}

    ===================================================

    AI SUMMARY:
    {summary if summary else "No summary available"}

    ===================================================
    Generated with AI Audio Processor
    """

    file_obj = io.BytesIO(content.encode('utf-8'))
    return file_obj, filename

# Step 6: Create interface
def create_interface():
    with gr.Blocks(title="🎤 AI Audio Processor", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🎤 AI Audio Processor
        **Ready to use! Models already loaded ✅**
        """)

        with gr.Row():
            with gr.Column():
                audio_input = gr.Audio(
                    sources=["upload", "microphone"],
                    type="filepath",
                    label="Upload Audio File"
                )

                language = gr.Dropdown(
                    choices=["en", "es", "fr", "de", "it", "pt", "ru", "zh", "ja", "ko"],
                    value="en",
                    label="Audio Language"
                )

                process_btn = gr.Button("🚀 Process Audio", variant="primary")

            with gr.Column():
                transcript_output = gr.Textbox(
                    label="📝 Full Transcript",
                    lines=12,
                    show_copy_button=True
                )

                summary_output = gr.Textbox(
                    label="📋 AI Summary",
                    lines=8,
                    show_copy_button=True
                )

                status_output = gr.Textbox(
                    label="Status",
                    interactive=False
                )

                with gr.Row():
                    download_btn = gr.Button("📥 Download Results", variant="secondary")
                    download_file = gr.File(
                        label="Download File",
                        visible=False
                    )

        process_btn.click(
            fn=process_audio,
            inputs=[audio_input, language],
            outputs=[transcript_output, summary_output, status_output]
        )

        download_btn.click(
            fn=download_results,
            inputs=[transcript_output, summary_output],
            outputs=[download_file, status_output]
        ).then(
            lambda: gr.File(visible=True),
            outputs=download_file
        )

    return demo

# Step 7: Main execution with auto port selection
if __name__ == "__main__":
    # Find available port
    available_port = find_available_port(7860)
    print(f"🔍 Found available port: {available_port}")

    # Create interface
    demo = create_interface()

    print("🚀 Launching AI Audio Processor...")

    # Launch with the available port
    try:
        public_url = demo.launch(share=True, server_port=available_port)
        print(f"🌐 Your public URL: {public_url}")

        from IPython.display import HTML
        display(HTML(f"""
        <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
                    padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
            <h2>🎉 Success! Audio Processor is Live!</h2>
            <p><strong>🌐 Public URL:</strong> <a href="{public_url}" target="_blank" style="color: #ffdd00; text-decoration: none;">{public_url}</a></p>
            <p><strong>⚡ Status:</strong> Models already loaded - Ready to process!</p>
            <p><strong>🎯 Port:</strong> {available_port} (automatically selected)</p>
        </div>
        """))

    except Exception as e:
        print(f"❌ Launch failed: {e}")
        print("🔄 Trying with default settings...")
        public_url = demo.launch(share=True)
        print(f"🌐 Backup URL: {public_url}")

✅ Models already loaded from previous run!
🔍 Reusing existing models to save time...


100%|███████████████████████████████████████| 461M/461M [00:10<00:00, 46.2MiB/s]


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/460M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/460M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


🎯 Models successfully reused!
🔍 Found available port: 7861
🚀 Launching AI Audio Processor...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://551e072bb4c06d97f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🌐 Your public URL: 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')